## Resource Constrained Project Scheduling

The Resource-Constrained Project Scheduling Problem (RCPSP) is a combinatorial optimization problem that consists of finding a feasible scheduling for a set of 𝑛 jobs subject to resource and precedence constraints. Each job has a processing time, a set of successors jobs and a required amount of different resources. Resources may be scarce but are renewable at each time period. Precedence constraints between jobs mean that no jobs may start before all its predecessors are completed. The jobs must be scheduled non-preemptively, i.e., once started, their processing cannot be interrupted.

The RCPSP has the following input data:


jobs set


renewable resources set


set of precedences between jobs (𝑖,𝑗)∈×


planning horizon: set of possible processing times for jobs

𝑝𝑗
processing time of job 𝑗

𝑢(𝑗,𝑟)
amount of resource 𝑟 required for processing job 𝑗

𝑐𝑟
capacity of renewable resource 𝑟

In addition to the jobs that belong to the project, the set  contains jobs 0 and 𝑛+1, which are dummy jobs that represent the beginning and the end of the planning, respectively. The processing time for the dummy jobs is always zero and these jobs do not consume resources.

A binary programming formulation was proposed by Pritsker et al. [PWW69]. In this formulation, decision variables 𝑥𝑗𝑡=1 if job 𝑗 is assigned to begin at time 𝑡; otherwise, 𝑥𝑗𝑡=0. All jobs must finish in a single instant of time without violating precedence constraints while respecting the amount of available resources. The model proposed by Pristker can be stated as follows:

$
\begin{align*}
&Minimize& &\sum_{t \in \tau} t * x_{(n+1, t)} \\
&s.t.& &\sum_{t \in \tau} x_{(j, t)} = 1 \forall{j} \in J \\
&& &\sum_{j \in J} \sum_{t_2 = t-p_j + 1}^t u_{(j,r)} x_{(j, t_2)} \leqq c_r \forall t \in \tau , r \in R \\
&& &\sum_{t \in \tau}t * x_{(x, t)} - \sum_{t \in \tau}t * x_{(j, t)} \geqq p_j \forall(j,s) \in S \\
&& &x_{(j, t)} \in {0, 1} \forall j \in J, t \in \tau \\
\end{align*}
$

An instance is shown below. The figure shows a graph where jobs in  are represented by nodes and precedence relations  are represented by directed edges. The time-consumption 𝑝𝑗 and all information concerning resource consumption 𝑢(𝑗,𝑟) are included next to the graph. This instance contains 10 jobs and 2 renewable resources, ={𝑟1,𝑟2}, where 𝑐1 = 6 and 𝑐2 = 8. Finally, a valid (but weak) upper bound on the time horizon  can be estimated by summing the duration of all jobs.

In [1]:
from itertools import product
from mip import Model, xsum, BINARY

n = 10  # note there will be exactly 12 jobs (n=10 jobs plus the two 'dummy' ones)

p = [0, 3, 2, 5, 4, 2, 3, 4, 2, 4, 6, 0]

u = [[0, 0], [5, 1], [0, 4], [1, 4], [1, 3], [3, 2], [3, 1], [2, 4],
     [4, 0], [5, 2], [2, 5], [0, 0]]

c = [6, 8]

S = [[0, 1], [0, 2], [0, 3], [1, 4], [1, 5], [2, 9], [2, 10], [3, 8], [4, 6],
     [4, 7], [5, 9], [5, 10], [6, 8], [6, 9], [7, 8], [8, 11], [9, 11], [10, 11]]

(R, J, T) = (range(len(c)), range(len(p)), range(sum(p)))

model = Model()

x = [[model.add_var(name="x({},{})".format(j, t), var_type=BINARY) for t in T] for j in J]

model.objective = xsum(t * x[n + 1][t] for t in T)

for j in J:
    model += xsum(x[j][t] for t in T) == 1

for (r, t) in product(R, T):
    model += (
        xsum(u[j][r] * x[j][t2] for j in J for t2 in range(max(0, t - p[j] + 1), t + 1))
        <= c[r])

for (j, s) in S:
    model += xsum(t * x[s][t] - t * x[j][t] for t in T) >= p[j]

model.optimize()

print("Schedule: ")
for (j, t) in product(J, T):
    if x[j][t].x >= 0.99:
        print("Job {}: begins at t={} and finishes at t={}".format(j, t, t+p[j]))
print("Makespan = {}".format(model.objective_value))

Welcome to the CBC MILP Solver 
Version: devel 
Build Date: Nov 15 2020 

Starting solution of the Linear programming relaxation problem using Primal Simplex

Coin0506I Presolve 100 (0) rows, 418 (-2) columns and 3850 (-2) elements
Clp1000I sum of infeasibilities 0.000128282 - average 1.28282e-06, 37 fixed columns
Coin0506I Presolve 100 (0) rows, 381 (-39) columns and 3648 (-204) elements
Clp0029I End of values pass after 381 iterations
Clp0000I Optimal - objective value 14
Clp0000I Optimal - objective value 14
Coin0511I After Postsolve, objective 14, infeasibilities - dual 0 (0), primal 0 (0)
Clp0000I Optimal - objective value 14
Clp0000I Optimal - objective value 14
Clp0000I Optimal - objective value 14
Clp0032I Optimal objective 14 - 0 iterations time 0.072, Idiot 0.07

Starting MIP optimization
Cgl0003I 35 fixed, 0 tightened bounds, 1 strengthened rows, 0 substitutions
Cgl0003I 2 fixed, 0 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0003I 10 fixed, 0 tightened bounds, 